In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc

import Qubits

EC = 0.04
EJ = 10
T = 0.01
ng = 0

In [2]:
transmonC = Qubits.transmon_charge(20, EC, EJ, ng)#Values given in Ghz*hbar

transmonC.solve()

gamma_arr = transmonC.T_1_gamma()
print(gamma_arr)

print(1/gamma_arr*1e6)


print(1/np.sum(gamma_arr)*1e6)

Charge matrix element: 0.2796437158192505
Phi matrix element: 0.08343335271674272
Qubit energies: -9.115688548244199 and -7.367785487915755
Qubit frequency: 1.7479030603284436
[2.51308743e+00 5.25883125e-01 7.48580171e+02]
[3.97916916e+05 1.90156321e+06 1.33586226e+03]
1330.4610601139948


In [3]:
transmonF = Qubits.transmon_flux(200, EC, EJ, ng)#Values given in Ghz*hbar

transmonF.solve()

gamma_arr_flux = transmonF.T_1_gamma()
print(gamma_arr_flux)

print(1/gamma_arr_flux)
print(1/np.sum(gamma_arr_flux)*1e6)

Charge matrix element: 0.2804534994734415
Phi matrix element: 0.08377092279477034
Qubit energies: -9.111596438479497 and -7.356274847647667
Qubit frequency: 1.75532159083183
[2.50971292e+00 5.29644401e-01 7.52556414e+02]
[3.98451948e-01 1.88805923e+00 1.32880404e-03]
1323.4589676815426


In [4]:
gatemonF = Qubits.gatemon_flux(200, EC, EJ, T, ng)

gatemonF.solve()

gamma_gatemonF = gatemonF.T_1_gamma()

print(gamma_gatemonF)

print(1/gamma_gatemonF)
print(1/np.sum(gamma_gatemonF)*1e6)

Charge matrix element: 0.1889400767225995
Phi matrix element: 0.44161577007782704
Qubit energies: -9.939836509679889 and -9.812422769895436
Qubit frequency: 0.12741373978445303
[2.32931220e+01 2.59004422e-02 2.96172962e+03]
[4.29311279e-02 3.86093795e+01 3.37640544e-04]
335.0029163652316


In [5]:
gatemonC = Qubits.gatemon_charge(200, EC, EJ, T, ng)

gatemonC.solve()

gamma_gatemonC = gatemonC.T_1_gamma()

print(gamma_gatemonC)

print(1/gamma_gatemonC)
print(1/np.sum(gamma_gatemonC)*1e6)

Charge matrix element: 0.019310962817451944
Phi matrix element: 0.4423523464599894
Qubit energies: -9.9400232699214 and -9.81247013409735
Qubit frequency: 0.12755313582405137
[2.37811397e+00 2.65009776e-03 2.96674785e+03]
[4.20501293e-01 3.77344570e+02 3.37069428e-04]
336.79915200960505
